In [ ]:
using IJulia, Plots, Printf

# Steady 1D Shallow Water equations

The purpose of this jupyter notebook is to provide a public and free usable stable implementation of the solution of the steady 1D SWE, first adopting a fixed bad approach and then moving to the mobile bed.

## Problem definition
First of all the definition of the problem is required, as well as the equations that will be solved.  
The problem consists in a (wide) rectangular channel, that can present steps, gates, localized narrowing or whatever along its longitudinal direction. The same solution could be performed for any prismatic channel, the choice of rectangular shape is done just for simplicity purposes. 
The situation in a section is depicted in the figure, at the same time the foundamental definitions referst to the image:  
  
![Problem description](images/problem_definition.svg)
  
The image shows a genelar section for which the following quantities are defined:
- d: water depth $[m]$;
- h: water level $[m]$;
- z: bed elevation (fixed) $[m]$;
- $kin:= \alpha \frac{U}{2g}$ $[m]$: the kinetic term of the energy;
- $H = h + kin$: the total Head (or Energy) $[m]$;
- $E = H - z = d + kin$: the Specific head (or Energy) $[m]$;
- $S = P_G \Omega + \rho \beta U^2 \Omega$: the Specific force [$N/m^2$].
  
Here $P_G$ is the pressure at the barycenter of the section, $\Omega$ is the cross-sectional area and $U$ the mean velocity all over the section.  



## The energy equations
As anticipated, the problem is solved through the energy equations, obtained from the momentum equation moltiplying them for the velocity $U$, and assuming the steady state. The equations reads as follows:  
$\frac{dh}{dx} = \frac{\beta \frac{F_r^2}{B} \frac{\partial \Omega}{\partial x}|_h - j}{1+\beta F_r^2}$  
  
$\frac{dd}{dx} = \frac{\beta \frac{F_r^2}{B} \frac{\partial \Omega}{\partial x}|_d + i_F - j}{1+\beta F_r^2}$  
  
$\frac{dH}{dx} = - j$  

$\frac{dE}{dx} = i_F - j$  
  
The new terms needs a brief explanation: $j$ is the friction term, which is defined as $j = \frac{\tau _0}{\rho g R_H}$, it contains the friction $\tau _0$, the water density $\rho$, the gravity term $g$, and the hydraulic radius $R_H = \Omega / P$, with $P$ the wetted perimeter. The term $\frac{\partial \Omega}{\partial x}|_h$ is related to the non prismaticity of the channel, describe how the wetted area changes according to the geometry. This last term is zero in case of completely rectangular channel, in any case for fixed geometry it is just a coefficient that can be precompute and just considered into the equations. Since the equation will be solved for rectangular channel further simplification may be performed, to do not loose generality it will not, the coefficient accounting for the section irregularity will just setted properly.

### Considerations before implementing everything
The codes are provided in Julia. It is important to know that any numerical method for solving ODE could be implemented, in this case some manipulations and considerations are adopted for speed in the computations, that is one of the objectivies of this work.  
The steps for the implementations are the following:
1) Build the geometry and define the $dx$;
2) Calculate the critical depth for any considered section;
3) Calculate the 2 normal depths for any section;
4) Calculate the Specific Force $S$ for both the depths for each section;
5) Assign the depth with the greatest $S$ between the two (disxrimination between fast/slow flows).
6) Include local discontinuities. 

In [28]:
# Build the geometry and define the $dx$;

@views function evaluate_uniform_depth(Q, B, Ks, iF)
    # Find the normal depth from the formula above, it is an implicit formula, so adopt an iterative procedure:
    Qnew = 0.0; tol = 1.0e-4; res = 0.1; n = 0; d = 1.0; # initial guess for the depth [m]
    initial_res = Q - B * d * Ks * ((B * d)/(2*d + B))^(2/3) * iF^(1/2);
    while abs(res) >= tol 
        Qnew = B * d * Ks * ((B * d)/(2*d + B))^(2/3) * iF^(1/2)
        res = Q - Qnew;
        increment = res/initial_res*0.1;
        if res > 0 # so the depth is too small
            d += increment; # increase the depth
        else
            d -= increment; # decrease the depth
        end
        n += 1
    end

    #= 
    println("Normal depth: ", d, " m, after ", n, " iterations.")
    println("Residual: ", res, " m^3/s, Discarge QNew: ", Qnew, " m^3/s")
    Yu = (Q/(B * Ks * iF^(1/2)))^(3/5)
    println("Normal depth WRC: ", Yu, " m")
    =#
    return d
end #function

L = 1000.0  # length of the channel [m]
Q = 20.0    # discharge [m^3/s] Q = Ω Ks R^2/3 iF^1/2
B = [100.0]    # width of the channel [m]
Ks = 40.0   # Strickler coefficient [m^(1/3)/s] Ks = 1/n
iF = [0.001]  # slope of the channel



n_segments = 1 # number of segments of the channel (with different slopes)
d_uniform = Array{Float64}(undef, n_segments) # array to store the uniform depths
for n in 1:n_segments
    d_uniform[n] = evaluate_uniform_depth(Q, B[n], Ks, iF[n])
end

println("Uniform depths: ", d_uniform, " m")




Uniform depths: [0.33153635556900923] m
